In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
PATH = "__tuning"

df = pd.read_csv(os.path.join(PATH, "QAResultData.csv"))
data = np.array(df.values)
df_keys = np.array([[eval(i)[0], eval(i)[1]] for i in data[:,0]])

 
logi_pool = [(file_name[0][:2] == 'M'+str(1))&(file_name[0][2] == 'M')&('SRM' not in file_name[0]) for file_name in df_keys] 

parameter = df_keys[logi_pool][:,1]
Ct_err = data[logi_pool][:,2]
Cc_err = data[logi_pool][:,3]
CD = data[logi_pool][:,4]   
SP = data[logi_pool][:,5]

train_X = np.array([[CD[i], SP[i]] for i in range(0, len(CD), 2)])
#(Ct_err1*wext2 - Ct_err2*wext1)/(Ct_err1 - Ct_err2)
train_Y = np.array([(Ct_err[i]*parameter[i+1][0]-Ct_err[i+1]*parameter[i][0])/(Ct_err[i]-Ct_err[i+1]) for i in range(0, len(Ct_err), 2)])

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


In [44]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

regressor = DecisionTreeRegressor(random_state=0, min_samples_leaf = 10)

In [45]:
regressor.fit(train_X, train_Y)

DecisionTreeRegressor(min_samples_leaf=10, random_state=0)

In [46]:
regressor.tree_.feature

array([ 0,  0,  1,  1, -2,  0,  1,  0, -2, -2, -2,  1, -2,  1, -2, -2,  0,
        1,  1, -2, -2, -2,  0,  0, -2,  0, -2, -2, -2,  1,  0,  0, -2,  1,
       -2, -2,  1,  0, -2, -2, -2, -2,  0,  1,  0, -2,  1, -2, -2,  0, -2,
        1,  1,  0, -2,  1, -2, -2, -2,  1, -2, -2,  1,  1, -2,  1, -2, -2,
        0,  1,  0, -2,  1,  1, -2, -2, -2,  1, -2,  1, -2, -2,  1,  0,  0,
       -2, -2, -2,  1,  1, -2, -2,  1, -2, -2], dtype=int64)

In [47]:
regressor.tree_.threshold

array([425. , 359. , 334.5, 125. ,  -2. , 305. , 224. , 260. ,  -2. ,
        -2. ,  -2. , 175.5,  -2. , 282.5,  -2. ,  -2. , 257. , 479. ,
       416. ,  -2. ,  -2. ,  -2. , 335. , 281. ,  -2. , 327. ,  -2. ,
        -2. ,  -2. , 472.5, 397. , 367. ,  -2. , 395. ,  -2. ,  -2. ,
       313. , 410. ,  -2. ,  -2. ,  -2. ,  -2. , 489. , 310.5, 453. ,
        -2. , 207.5,  -2. ,  -2. , 435. ,  -2. , 483. , 442.5, 452. ,
        -2. , 379. ,  -2. ,  -2. ,  -2. , 519. ,  -2. ,  -2. , 302. ,
       138. ,  -2. , 263.5,  -2. ,  -2. , 545. , 439.5, 499. ,  -2. ,
       339. , 323.5,  -2. ,  -2. ,  -2. , 450.5,  -2. , 522. ,  -2. ,
        -2. , 433.5, 569. , 559. ,  -2. ,  -2. ,  -2. , 471.5, 453.5,
        -2. ,  -2. , 559. ,  -2. ,  -2. ])

In [48]:
regressor.tree_.children_left

array([ 1,  2,  3,  4, -1,  6,  7,  8, -1, -1, -1, 12, -1, 14, -1, -1, 17,
       18, 19, -1, -1, -1, 23, 24, -1, 26, -1, -1, -1, 30, 31, 32, -1, 34,
       -1, -1, 37, 38, -1, -1, -1, -1, 43, 44, 45, -1, 47, -1, -1, 50, -1,
       52, 53, 54, -1, 56, -1, -1, -1, 60, -1, -1, 63, 64, -1, 66, -1, -1,
       69, 70, 71, -1, 73, 74, -1, -1, -1, 78, -1, 80, -1, -1, 83, 84, 85,
       -1, -1, -1, 89, 90, -1, -1, 93, -1, -1], dtype=int64)

In [49]:
regressor.tree_.children_right

array([42, 29, 16,  5, -1, 11, 10,  9, -1, -1, -1, 13, -1, 15, -1, -1, 22,
       21, 20, -1, -1, -1, 28, 25, -1, 27, -1, -1, -1, 41, 36, 33, -1, 35,
       -1, -1, 40, 39, -1, -1, -1, -1, 62, 49, 46, -1, 48, -1, -1, 51, -1,
       59, 58, 55, -1, 57, -1, -1, -1, 61, -1, -1, 68, 65, -1, 67, -1, -1,
       82, 77, 72, -1, 76, 75, -1, -1, -1, 79, -1, 81, -1, -1, 88, 87, 86,
       -1, -1, -1, 92, 91, -1, -1, 94, -1, -1], dtype=int64)

In [50]:
len(regressor.tree_.children_right)

95

In [51]:
regressor.tree_.value[0][0]

array([-7.25082886])

In [52]:
def split(array, dim, thred):
    if dim == 0:
        return [[np.min(array[0]), thred], array[1]],  [[thred, np.max(array[0])], array[1]]
    elif dim == 1:
        return [array[0], [np.min(array[1]), thred]],  [array[0], [thred, np.max(array[1])]]

In [53]:
interval = [[0,1000], [0,1000]]
left_children = [interval]
right_children = [interval]
right_remove_idx = False
left_remove_idx = False
parent = []
region = []
LEFT_FLAG = False

for index_of_node in range(0,len(regressor.tree_.threshold)):
    if regressor.tree_.threshold[index_of_node] != -2:
        
#         print(right_children)
        
        if LEFT_FLAG == False:
            
            FROM_LEFT_FLAG = False
            
            right, left = split(right_children[-1], regressor.tree_.feature[index_of_node], regressor.tree_.threshold[index_of_node])
            
            parent.append([0,1,len(right_children)-1, False, False])
            right_children.append(right)
            left_children.append(left)
#             print(left_children)
            FROM_RIGHT_FLAG = True
        else:
            FROM_RIGHT_FLAG = False
            
            right, left = split(left_children[-1], regressor.tree_.feature[index_of_node], regressor.tree_.threshold[index_of_node])
            
            
            
            parent.append([1, 0,len(left_children)-1, False, False])
            
            right_children.append(right)
            left_children.append(left)
            LEFT_FLAG = False
            FROM_LEFT_FLAG = True
        
    else:
        
        
        if regressor.tree_.threshold[index_of_node-1] != -2:
            sub_region = right_children[-1]
            region.append([sub_region[0][0], sub_region[0][1], sub_region[1][0], sub_region[1][1], regressor.tree_.value[index_of_node][0][0]])
            
            parent_idx = parent[-1][0:3]
#             parent_remove = parent[-1][3:]
            
            right_children.pop()
            LEFT_FLAG = True
#             parent_remove[1] = True
            parent[-1][4] = True
        else:
            sub_region = left_children[-1]
            region.append([sub_region[0][0], sub_region[0][1], sub_region[1][0], sub_region[1][1], regressor.tree_.value[index_of_node][0][0]])
            
            parent_idx = parent[-1][0:3]
#             parent_remove = parent[-1][3:]

            left_children.pop()
            parent[-1][3] = True
     
    
#         print(left_remove_idx, right_remove_idx)
        while (parent[-1][3:][0] == True) & (parent[-1][3:][1] == True):
#             if len(parent == len(regressor.tree_.threshold)-1:
#                 break:
            
            if parent_idx[0] == 1: #left
                left_children.pop(parent_idx[2])
                parent.pop()
                if (index_of_node == len(regressor.tree_.feature)-1) & (len(parent) == 0):
                    break
                else:
                    parent_idx = parent[-1][0:3]
#                 parent_remove = parent[-1][3:]
                
                    parent[-1][3] = True
            else:
                right_children.pop(parent_idx[2])
                parent.pop()
                if (index_of_node == len(regressor.tree_.feature)-1) & (len(parent) == 0):
                    break
                else:
                    parent_idx = parent[-1][0:3]
    #                 parent_remove = parent[-1][3:]

                    parent[-1][4] = True

        else:
            continue


In [54]:
right_children

[]

In [55]:
left_children

[[[0, 1000], [0, 1000]]]

In [56]:
print(tree.export_text(regressor))

|--- feature_0 <= 425.00
|   |--- feature_0 <= 359.00
|   |   |--- feature_1 <= 334.50
|   |   |   |--- feature_1 <= 125.00
|   |   |   |   |--- value: [-4.74]
|   |   |   |--- feature_1 >  125.00
|   |   |   |   |--- feature_0 <= 305.00
|   |   |   |   |   |--- feature_1 <= 224.00
|   |   |   |   |   |   |--- feature_0 <= 260.00
|   |   |   |   |   |   |   |--- value: [5.11]
|   |   |   |   |   |   |--- feature_0 >  260.00
|   |   |   |   |   |   |   |--- value: [2.74]
|   |   |   |   |   |--- feature_1 >  224.00
|   |   |   |   |   |   |--- value: [1.40]
|   |   |   |   |--- feature_0 >  305.00
|   |   |   |   |   |--- feature_1 <= 175.50
|   |   |   |   |   |   |--- value: [-3.76]
|   |   |   |   |   |--- feature_1 >  175.50
|   |   |   |   |   |   |--- feature_1 <= 282.50
|   |   |   |   |   |   |   |--- value: [2.67]
|   |   |   |   |   |   |--- feature_1 >  282.50
|   |   |   |   |   |   |   |--- value: [0.04]
|   |   |--- feature_1 >  334.50
|   |   |   |--- feature_0 <= 257.00


In [57]:
region

[[0.0, 359.0, 0.0, 125.0, -4.738047953782854],
 [0.0, 260.0, 125.0, 224.0, 5.110972969698491],
 [260.0, 305.0, 125.0, 224.0, 2.740231589505737],
 [0.0, 305.0, 224.0, 334.5, 1.4037981694167485],
 [305.0, 359.0, 125.0, 175.5, -3.7648234441744757],
 [305.0, 359.0, 175.5, 282.5, 2.6728977324270224],
 [305.0, 359.0, 282.5, 334.5, 0.041581970247154154],
 [0.0, 257.0, 334.5, 416.0, -1.3249233104912626],
 [0.0, 257.0, 416.0, 479.0, -0.5786981966181725],
 [0.0, 257.0, 479.0, 1000.0, -3.7610006941268552],
 [257.0, 281.0, 334.5, 1000, -4.2401094099659],
 [281.0, 327.0, 334.5, 1000, -6.913856868386265],
 [327.0, 335.0, 334.5, 1000, -5.090116062566234],
 [335.0, 359.0, 334.5, 1000, -1.0418604937999556],
 [359.0, 367.0, 0, 472.5, -4.7270644376361455],
 [367.0, 397.0, 0.0, 395.0, -1.4550026577973476],
 [367.0, 397.0, 395.0, 472.5, -3.008780865753251],
 [397.0, 410.0, 0.0, 313.0, -2.807341927796451],
 [410.0, 425.0, 0.0, 313.0, -5.431124607961797],
 [397.0, 425.0, 313.0, 472.5, -7.163188648866617],
 [

In [59]:
header = [['CD1', 'CD2', 'SP1', 'SP2', 'wext']]
import csv

with open('rules.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(header)
    writer.writerows(region)

In [61]:
with open('rules.csv', 'r', newline='') as file:
    rows = csv.reader(file)
#     rows = csv.reader(csvfile)

  # 以迴圈輸出每一列
    